In [ ]:
import pandas as pd
import numpy as np
import math
import subprocess
from datetime import date, time, datetime, timedelta
from IPython.display import display
import calendar
import bandit.main as bd

In [ ]:
def df_bandit_class(df, classname):
    dfgb = df.groupby([classname])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [ ]:
def df_search(df_in, cohort):
    if not cohort in df_in:
        return df_in

    df_class = df_bandit_class(df_in, cohort)
    display(df_class)
    next_class = df_class.iloc[0, 0]
    if df_in['Complete'].isnull().sum() > 0:
        df_filtered = df_in[df_in[cohort] == next_class]
    else:
        df_filtered = df_in

    return df_filtered

In [ ]:
def rate_calc(df):
    pedal_time = df['Goal'] * df['seconds']
    rest_time = (df['Goal'] - 1) * 15
    total_time = pedal_time + rest_time
    rate = pedal_time / total_time
    score = pedal_time * rate
    return int(score)

In [ ]:
currDate = datetime.today()
goal_list = [[x, y] for x in range(1,15) for y in [72,76,80,84,88]]
df = pd.DataFrame.from_records(goal_list).rename(index=str, columns={0: "Minutes", 1: "Rate"})
df['Date'] = currDate
df['Complete'] = np.nan
df = df[['Date','Minutes','Rate','Complete']]

In [ ]:
df_excel = pd.read_excel(io='PunchGoals.xlsx')
foo = df_excel.groupby(['Minutes','Rate']).count().reset_index()
display(foo)
foo['remove'] = np.where(foo['Complete'] > 0, True, False)
foo = foo[['Minutes','Rate','remove']]
df = df.merge(foo,on=['Minutes','Rate'], how='left')
df = df[df['remove'] != True].drop(['remove'],axis=1)
df

In [ ]:
subprocess.Popen('PunchGoals.xlsx', shell=True)

In [ ]:
df = df.append(df_excel).reset_index(drop=True)

In [ ]:
df['Value'] = df['Minutes'] * (df['Rate'] - 10)
val_df = bd.set_cohorts(df, 'Value')
df = df.merge(val_df, on='Value')
df['Score'] = np.where(df['Complete'] >= df['Minutes'], df['Value']**1.2, df['Value'] / 5)
df.loc[df['Complete'].isnull(), 'Score'] = np.NaN
df['Class'] = df['Minutes'].map(str) + '-' + df['Rate'].map(str)
df = bd.reduce(df,'Class')

In [ ]:
df_quality = bd.get_quality(df['Score'])
df = df.join(df_quality)

In [ ]:
df_step1 = df_search(df,'cohort1')
df_step1

In [ ]:
df_step2 = df_search(df_step1,'cohort2')
df_step2

In [ ]:
df_step3 = df_search(df_step2,'cohort3')
df_step3

In [ ]:
df_bandit_class(df_step3, 'Class')